In [2]:
import datetime
import numpy as np
import pandas as pd

# 20190827

In [14]:
c14_patient_data = pd.read_excel('../../../data/C14data_liver_samples_20190827.xlsx')[
    ['sort', 'Code', 'sample', 'DOB', 'DOA', u'Δ 14C', 'Error, 2 s', 'no sorted', 'Pathology', 'sex']]
c14_patient_data.columns = ['type', 'code', 'sample', 'Dbirth', 'Dcoll', 'd14C', 'e14C', 'N_cells', 'pathology', 'sex']

c14_patient_data['d14C'] /= 1000.0
c14_patient_data['e14C'] /= 1000.0 # convert unit
c14_patient_data['e14C'] /= 2.0 # convert to 1s error
c14_patient_data['subject_age'] = c14_patient_data['Dcoll'] - c14_patient_data['Dbirth']

def convert_N_cell_entry(x):
    if isinstance(x, float):
        return x
    elif x[-1] == '2':
        return float(x.rstrip(' mio /2')) / 2
    else:
        return float(x.rstrip(' mio'))

c14_patient_data['N_cells'] = c14_patient_data['N_cells'].apply(convert_N_cell_entry)

c14_patient_data.loc[c14_patient_data['type'] == 'HNF4α -', 'type'] = 'non-hepatocyte'

c14_patient_data['ploidy'] = '2n4n'
for type_ in ['2n', '4n']:
    c14_patient_data.loc[c14_patient_data['type'].str.startswith(type_), 'ploidy'] = type_
    c14_patient_data.loc[c14_patient_data['type'].str.startswith(type_), 'type'] = 'hepatocyte'


c14_patient_data['type'] = pd.Categorical(c14_patient_data['type'])
c14_patient_data['code'] = pd.Categorical(c14_patient_data['code'])
c14_patient_data['sample'] = pd.Categorical(c14_patient_data['sample'])
c14_patient_data['pathology'] = pd.Categorical(c14_patient_data['pathology'])

c14_patient_data.head()

,type,code,sample,Dbirth,Dcoll,d14C,e14C,N_cells,pathology,sex,subject_age,ploidy
0,unsorted,PH1,ND380,1942.833333,2016.416667,0.078416,0.003981,NaN,N,F,73.583333,2n4n
1,unsorted,PH2,ND388,1991.416667,2016.833333,0.025254,0.003912,NaN,N,M,25.416667,2n4n
2,hepatocyte,PH6,ND022,1946.500000,2004.833333,0.075001,0.004711,62.0,N,M,58.333333,2n4n
3,hepatocyte,PH7,ND023,1945.833333,2004.833333,0.090351,0.004395,70.0,N,M,59.000000,2n4n
4,hepatocyte,PH11,ND375,1992.500000,2016.333333,0.003700,0.016000,61.0,N,M,23.833333,2n4n


Drop the patient with unknown age

In [15]:
c14_patient_data = c14_patient_data[c14_patient_data['Dbirth'].notna()]

In [16]:
c14_patient_data.to_csv(f'../../../results/C14data_liver_samples_20190827.csv', index=False)
c14_patient_data.to_pickle(f'../../../results/C14data_liver_samples_20190827.pkl')

In [17]:
pd.read_pickle(f'../../../results/C14data_liver_samples_20190827.pkl').head()

,type,code,sample,Dbirth,Dcoll,d14C,e14C,N_cells,pathology,sex,subject_age,ploidy
0,unsorted,PH1,ND380,1942.833333,2016.416667,0.078416,0.003981,NaN,N,F,73.583333,2n4n
1,unsorted,PH2,ND388,1991.416667,2016.833333,0.025254,0.003912,NaN,N,M,25.416667,2n4n
2,hepatocyte,PH6,ND022,1946.500000,2004.833333,0.075001,0.004711,62.0,N,M,58.333333,2n4n
3,hepatocyte,PH7,ND023,1945.833333,2004.833333,0.090351,0.004395,70.0,N,M,59.000000,2n4n
4,hepatocyte,PH11,ND375,1992.500000,2016.333333,0.003700,0.016000,61.0,N,M,23.833333,2n4n


# 20190722

In [5]:
c14_patient_data = pd.read_excel('../../../data/C14data_liver_samples_20190722.xlsx')[
    ['sort', 'Code', 'sample', 'DOB', 'DOA', u'Δ 14C', 'Error, 2 s', 'no sorted', 'Pathology']]
c14_patient_data.columns = ['type', 'code', 'sample', 'Dbirth', 'Dcoll', 'd14C', 'error_measurement', 'N_cells', 'pathology']

c14_patient_data['d14C'] /= 1000.0
c14_patient_data['error_measurement'] /= 1000.0 # convert unit
c14_patient_data['error_measurement'] /= 2.0 # convert to 1s error
c14_patient_data['subject_age'] = c14_patient_data['Dcoll'] - c14_patient_data['Dbirth']

def convert_N_cell_entry(x):
    if isinstance(x, float):
        return x
    elif x[-1] == '2':
        return float(x.rstrip(' mio /2')) / 2
    else:
        return float(x.rstrip(' mio'))

c14_patient_data['N_cells'] = c14_patient_data['N_cells'].apply(convert_N_cell_entry)

c14_patient_data.loc[c14_patient_data['type'] == 'HNF4α -', 'type'] = 'non-hepatocyte'

c14_patient_data['ploidy'] = '2n4n'
for type_ in ['2n', '4n']:
    c14_patient_data.loc[c14_patient_data['type'].str.startswith(type_), 'ploidy'] = type_
    c14_patient_data.loc[c14_patient_data['type'].str.startswith(type_), 'type'] = 'hepatocyte'


c14_patient_data['type'] = pd.Categorical(c14_patient_data['type'])
c14_patient_data['code'] = pd.Categorical(c14_patient_data['code'])
c14_patient_data['sample'] = pd.Categorical(c14_patient_data['sample'])
c14_patient_data['pathology'] = pd.Categorical(c14_patient_data['pathology'])

c14_patient_data.head()

,type,code,sample,Dbirth,Dcoll,d14C,error_measurement,N_cells,pathology,subject_age,ploidy
0,unsorted,PH1,ND380,1942.833333,2016.416667,0.078416,0.003981,NaN,N,73.583333,2n4n
1,unsorted,PH2,ND388,1991.416667,2016.833333,0.025254,0.003912,NaN,N,25.416667,2n4n
2,hepatocyte,PH6,ND022,1946.500000,2004.833333,0.075001,0.004711,62.0,N,58.333333,2n4n
3,hepatocyte,PH7,ND023,1945.833333,2004.833333,0.090351,0.004395,70.0,N,59.000000,2n4n
4,hepatocyte,PH11,ND375,1992.500000,2016.333333,0.003700,0.016000,61.0,N,23.833333,2n4n


Drop the patient with unknown age

In [7]:
c14_patient_data = c14_patient_data[c14_patient_data['Dbirth'].notna()]

In [9]:
c14_patient_data.to_csv(f'../../../results/C14data_liver_samples_20190722.csv', index=False)
c14_patient_data.to_pickle(f'../../../results/C14data_liver_samples_20190722.pkl')

In [12]:
pd.read_pickle(f'../../../results/C14data_liver_samples_20190722.pkl').head()

,type,code,sample,Dbirth,Dcoll,d14C,error_measurement,N_cells,pathology,subject_age,ploidy
0,unsorted,PH1,ND380,1942.833333,2016.416667,0.078416,0.003981,NaN,N,73.583333,2n4n
1,unsorted,PH2,ND388,1991.416667,2016.833333,0.025254,0.003912,NaN,N,25.416667,2n4n
2,hepatocyte,PH6,ND022,1946.500000,2004.833333,0.075001,0.004711,62.0,N,58.333333,2n4n
3,hepatocyte,PH7,ND023,1945.833333,2004.833333,0.090351,0.004395,70.0,N,59.000000,2n4n
4,hepatocyte,PH11,ND375,1992.500000,2016.333333,0.003700,0.016000,61.0,N,23.833333,2n4n


# 20190502

In [16]:
c14_patient_data = pd.read_excel('../../data/C14data_liver_samples_20190502.xlsx')[
    ['sort', 'Code', 'sample', 'DOB', 'DOA', u'Δ 14C', 'Error, 2 s', 'no sorted', 'Pathology']]
c14_patient_data.columns = ['type', 'code', 'sample', 'Dbirth', 'Dcoll', 'd14C', 'error_measurement', 'N_cells', 'pathology']

c14_patient_data['d14C'] /= 1000.0
c14_patient_data['error_measurement'] /= 1000.0 # convert unit
c14_patient_data['error_measurement'] /= 2.0 # convert to 1s error
c14_patient_data['subject_age'] = c14_patient_data['Dcoll'] - c14_patient_data['Dbirth']

def convert_N_cell_entry(x):
    if isinstance(x, float):
        return x
    elif x[-1] == '2':
        return float(x.rstrip(' mio /2')) / 2
    else:
        return float(x.rstrip(' mio'))

c14_patient_data['N_cells'] = c14_patient_data['N_cells'].apply(convert_N_cell_entry)

c14_patient_data.loc[c14_patient_data['type'] == 'HNF4α -', 'type'] = 'non-hepatocyte'

c14_patient_data['ploidy'] = '2n4n'
for type_ in ['2n', '4n']:
    c14_patient_data.loc[c14_patient_data['type'].str.startswith(type_), 'ploidy'] = type_
    c14_patient_data.loc[c14_patient_data['type'].str.startswith(type_), 'type'] = 'hepatocyte'


c14_patient_data['type'] = pd.Categorical(c14_patient_data['type'])
c14_patient_data['code'] = pd.Categorical(c14_patient_data['code'])
c14_patient_data['sample'] = pd.Categorical(c14_patient_data['sample'])
c14_patient_data['pathology'] = pd.Categorical(c14_patient_data['pathology'])

c14_patient_data.head()

,type,code,sample,Dbirth,Dcoll,d14C,error_measurement,N_cells,pathology,subject_age,ploidy
0,unsorted,PH1,ND380,1942.833333,2016.416667,0.078416,0.003981,NaN,N,73.583333,2n4n
1,unsorted,PH2,ND388,1991.416667,2016.833333,0.025254,0.003912,NaN,N,25.416667,2n4n
2,hepatocyte,PH3,ND381,1985.166667,2016.416667,0.040336,0.008788,58.0,N,31.250000,2n4n
3,hepatocyte,PH4,ND382,1982.083333,2016.500000,0.018815,0.005342,52.0,N,34.416667,2n4n
4,hepatocyte,PH6,ND022,1946.500000,2004.833333,0.075001,0.004711,62.0,N,58.333333,2n4n


Drop the patient with unknown age

In [17]:
c14_patient_data = c14_patient_data[c14_patient_data['Dbirth'].notna()]

In [18]:
c14_patient_data.to_csv(f'../../results/C14data_liver_samples_20190502.csv', index=False)
c14_patient_data.to_pickle(f'../../results/C14data_liver_samples_20190502.pkl')

In [19]:
pd.read_csv(f'../../results/C14data_liver_samples_20190502.csv')['type'].head()

0      unsorted
1      unsorted
2    hepatocyte
3    hepatocyte
4    hepatocyte
Name: type, dtype: object

In [20]:
pd.read_pickle(f'../../results/C14data_liver_samples_20190502.pkl')['type'].head()

0      unsorted
1      unsorted
2    hepatocyte
3    hepatocyte
4    hepatocyte
Name: type, dtype: category
Categories (3, object): [hepatocyte, non-hepatocyte, unsorted]

In [21]:
pd.read_pickle(f'../../results/C14data_liver_samples_20190502.pkl').head()

,type,code,sample,Dbirth,Dcoll,d14C,error_measurement,N_cells,pathology,subject_age,ploidy
0,unsorted,PH1,ND380,1942.833333,2016.416667,0.078416,0.003981,NaN,N,73.583333,2n4n
1,unsorted,PH2,ND388,1991.416667,2016.833333,0.025254,0.003912,NaN,N,25.416667,2n4n
2,hepatocyte,PH3,ND381,1985.166667,2016.416667,0.040336,0.008788,58.0,N,31.250000,2n4n
3,hepatocyte,PH4,ND382,1982.083333,2016.500000,0.018815,0.005342,52.0,N,34.416667,2n4n
4,hepatocyte,PH6,ND022,1946.500000,2004.833333,0.075001,0.004711,62.0,N,58.333333,2n4n


# Old

In [ ]:
# # 20180505

# c14_patient_data = pd.read_excel('../../data/C14data_liver_samples_20180505.xlsx', 
#                                  usecols=[0, 1, 2, 3, 5, 6, 8, 9, 10])[
#     ['sort', 'Code', 'sample', 'DOB', 'DOA', u'Δ 14C', 'Error, 2 s', 'no sorted', 'Pathology']]
# c14_patient_data.columns = ['type', 'code', 'sample', 'Dbirth', 'Dcoll', 'd14C', 'error_measurement', 'N_cells', 'pathology']

# c14_patient_data['d14C'] /= 1000.0
# c14_patient_data['error_measurement'] /= 1000.0 # convert unit
# c14_patient_data['error_measurement'] /= 2.0 # convert to 1s error
# c14_patient_data['subject_age'] = c14_patient_data['Dcoll'] - c14_patient_data['Dbirth']

# invalid_codes = ['PH14', 'PH20', 'PH45', 'PH46']
# c14_patient_data['valid'] = np.logical_not(c14_patient_data['code'].isin(invalid_codes))

# c14_patient_data['type'] = pd.Categorical(c14_patient_data['type'])
# c14_patient_data['code'] = pd.Categorical(c14_patient_data['code'])
# c14_patient_data['sample'] = pd.Categorical(c14_patient_data['sample'])
# c14_patient_data['pathology'] = pd.Categorical(c14_patient_data['pathology'])

# def convert_N_cell_entry(x):
#     if isinstance(x, float):
#         return x
#     elif x[-1] == '2':
#         return float(x.rstrip(' mio /2')) / 2
#     else:
#         return float(x.rstrip(' mio'))

# c14_patient_data['N_cells'] = c14_patient_data['N_cells'].apply(convert_N_cell_entry)

# c14_patient_data.loc[c14_patient_data['type'] == 'HNF4α -', 'type'] = 'non-hepatocyte'
# c14_patient_data['type'].cat.remove_unused_categories(inplace=True)
    
# c14_patient_data.head()

# c14_patient_data.to_csv(f'../../results/C14data_liver_samples_{datetime.date.today().isoformat()}.csv', index=False)
# c14_patient_data.to_pickle(f'../../results/C14data_liver_samples_{datetime.date.today().isoformat()}.pkl')

# pd.read_csv('../../results/C14data_liver_samples_2018-12-05.csv')['type'].head()

# pd.read_pickle('../../results/C14data_liver_samples_2018-12-05.pkl')['type'].head()

# # 20190307

# c14_patient_data = pd.read_excel('../../data/C14data_liver_samples_20190307.xlsx', 
#                                  usecols=[0, 1, 2, 3, 5, 6, 8, 9, 10])[
#     ['sort', 'Code', 'sample', 'DOB', 'DOA', u'Δ 14C', 'Error, 2 s', 'no sorted', 'Pathology']]
# c14_patient_data.columns = ['type', 'code', 'sample', 'Dbirth', 'Dcoll', 'd14C', 'error_measurement', 'N_cells', 'pathology']

# c14_patient_data['d14C'] /= 1000.0
# c14_patient_data['error_measurement'] /= 1000.0 # convert unit
# c14_patient_data['error_measurement'] /= 2.0 # convert to 1s error
# c14_patient_data['subject_age'] = c14_patient_data['Dcoll'] - c14_patient_data['Dbirth']

# # invalid_codes = ['PH14', 'PH20', 'PH45', 'PH46']
# # c14_patient_data['valid'] = np.logical_not(c14_patient_data['code'].isin(invalid_codes))

# c14_patient_data['type'] = pd.Categorical(c14_patient_data['type'])
# c14_patient_data['code'] = pd.Categorical(c14_patient_data['code'])
# c14_patient_data['sample'] = pd.Categorical(c14_patient_data['sample'])
# c14_patient_data['pathology'] = pd.Categorical(c14_patient_data['pathology'])

# def convert_N_cell_entry(x):
#     if isinstance(x, float):
#         return x
#     elif x[-1] == '2':
#         return float(x.rstrip(' mio /2')) / 2
#     else:
#         return float(x.rstrip(' mio'))

# c14_patient_data['N_cells'] = c14_patient_data['N_cells'].apply(convert_N_cell_entry)

# c14_patient_data.loc[c14_patient_data['type'] == 'HNF4α -', 'type'] = 'non-hepatocyte'
# c14_patient_data['type'].cat.remove_unused_categories(inplace=True)
    
# c14_patient_data.head()

# Drop the patient with unknown age

# c14_patient_data[c14_patient_data.drop('N_cells', axis=1).isna().any(axis='columns')]

# c14_patient_data = c14_patient_data[c14_patient_data['Dbirth'].notna()]

# c14_patient_data.to_csv(f'../../results/C14data_liver_samples_{datetime.date.today().isoformat()}.csv', index=False)
# c14_patient_data.to_pickle(f'../../results/C14data_liver_samples_{datetime.date.today().isoformat()}.pkl')

# pd.read_csv('../../results/C14data_liver_samples_2019-03-22.csv')['type'].head()

# pd.read_pickle('../../results/C14data_liver_samples_2019-03-22.pkl')['type'].head()

# # 20190326

# pd.read_excel('../../data/C14data_liver_samples_20190326.xlsx').head()

# c14_patient_data = pd.read_excel('../../data/C14data_liver_samples_20190326.xlsx')[
#     ['sort', 'Code', 'sample', 'DOB', 'DOA', u'Δ 14C', 'Error, 2 s', 'no sorted', 'Pathology']]
# c14_patient_data.columns = ['type', 'code', 'sample', 'Dbirth', 'Dcoll', 'd14C', 'error_measurement', 'N_cells', 'pathology']

# c14_patient_data['d14C'] /= 1000.0
# c14_patient_data['error_measurement'] /= 1000.0 # convert unit
# c14_patient_data['error_measurement'] /= 2.0 # convert to 1s error
# c14_patient_data['subject_age'] = c14_patient_data['Dcoll'] - c14_patient_data['Dbirth']

# def convert_N_cell_entry(x):
#     if isinstance(x, float):
#         return x
#     elif x[-1] == '2':
#         return float(x.rstrip(' mio /2')) / 2
#     else:
#         return float(x.rstrip(' mio'))

# c14_patient_data['N_cells'] = c14_patient_data['N_cells'].apply(convert_N_cell_entry)

# c14_patient_data.loc[c14_patient_data['type'] == 'HNF4α -', 'type'] = 'non-hepatocyte'

# c14_patient_data['ploidy'] = '2n4n'
# for type_ in ['2n', '4n']:
#     c14_patient_data.loc[c14_patient_data['type'].str.startswith(type_), 'ploidy'] = type_
#     c14_patient_data.loc[c14_patient_data['type'].str.startswith(type_), 'type'] = 'hepatocyte'


# c14_patient_data['type'] = pd.Categorical(c14_patient_data['type'])
# c14_patient_data['code'] = pd.Categorical(c14_patient_data['code'])
# c14_patient_data['sample'] = pd.Categorical(c14_patient_data['sample'])
# c14_patient_data['pathology'] = pd.Categorical(c14_patient_data['pathology'])

# c14_patient_data.head()

# Drop the patient with unknown age

# c14_patient_data = c14_patient_data[c14_patient_data['Dbirth'].notna()]

# c14_patient_data.to_csv(f'../../results/C14data_liver_samples_{datetime.date.today().isoformat()}.csv', index=False)
# c14_patient_data.to_pickle(f'../../results/C14data_liver_samples_{datetime.date.today().isoformat()}.pkl')

# pd.read_csv('../../results/C14data_liver_samples_2019-04-05.csv')['type'].head()

# pd.read_pickle('../../results/C14data_liver_samples_2019-04-05.pkl')['type'].head()

# pd.read_pickle('../../results/C14data_liver_samples_2019-04-05.pkl').head()

